In [1]:
using Revise

In [2]:
using Polyhedra
using CDDLib
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyDynamics
using StaticArrays
using RigidBodyTreeInspector
using MechanismComplementarity
using JuMP, Gurobi

In [3]:
function DrakeVisualizer.addgeometry!(vis::Visualizer, obs::Obstacle, boundary::HRepresentation=SimpleHRepresentation(vcat(eye(3), -eye(3)), vcat([1., 0.5, 2.0], -[-1., -0.5, -0.1])))
    p = intersect(boundary, obs.interior)
    addgeometry!(vis, CDDPolyhedron{3, Float64}(p))
end

In [4]:
urdf_mech = parse_urdf(Float64, "box.urdf")
mechanism, base = planar_revolute_base()
attach!(mechanism, base, urdf_mech)
world = root_body(mechanism)

basevis = Visualizer()[:boxval]
delete!(basevis)
vis = basevis[:robot]
setgeometry!(vis, mechanism, parse_urdf("box_valkyrie.urdf", mechanism))

floor = planar_obstacle(default_frame(world), [0, 0, 1.], [0, 0, 0.])
free_space = space_between([floor])

addgeometry!(basevis[:environment], floor)

core = findbody(mechanism, "core")

env = Environment(
    Dict(core => ContactEnvironment(
                [
                Point3D(default_frame(core), SVector(0.1, 0, 0.2)),
                Point3D(default_frame(core), SVector(-0.1, 0, 0.2)),
                Point3D(default_frame(core), SVector(0.1, 0, -0.2)),
                Point3D(default_frame(core), SVector(-0.1, 0, -0.2)),
                 ],
                [floor],
                [free_space])))

MechanismComplementarity.Environment{Float64}(Dict(RigidBody: "core"=>MechanismComplementarity.ContactEnvironment{Float64}(RigidBodyDynamics.Point3D{SVector{3,Float64}}[Point3D in "after_core_to_world": [0.1, 0.0, 0.2], Point3D in "after_core_to_world": [-0.1, 0.0, 0.2], Point3D in "after_core_to_world": [0.1, 0.0, -0.2], Point3D in "after_core_to_world": [-0.1, 0.0, -0.2]], MechanismComplementarity.Obstacle{Float64}[MechanismComplementarity.Obstacle{Float64}(CartesianFrame3D: "world" (id = 5), H-representation
begin
 1 4 real
 0.0 -0.0 -0.0 -1.0
end, Polyhedra.HalfSpace{3,Float64}([0.0, 0.0, 1.0], 0.0), 1.0)], MechanismComplementarity.FreeRegion{Float64}[MechanismComplementarity.FreeRegion{Float64}(CartesianFrame3D: "world" (id = 5), H-representation
begin
 1 4 real
 -0.0 0.0 0.0 1.0
end)])))

In [7]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "base_x"), [-1])
set_configuration!(x0, findjoint(mechanism, "base_z"), [0.3])
set_configuration!(x0, findjoint(mechanism, "base_rotation"), [0.3])
set_velocity!(x0, findjoint(mechanism, "base_x"), [3])

settransform!(vis, x0)

controller = x -> begin
    zeros(num_velocities(x))
end

Δt = 0.05

results = MechanismComplementarity.simulate(x0, controller, env, Δt, 100, GurobiSolver(OutputFlag=0));

In [8]:
for r in results
    set_configuration!(x0, configuration(r.state))
    set_velocity!(x0, velocity(r.state))
    settransform!(vis, x0)
    sleep(Δt)
end